In [2]:
import numpy as np
import torch
import torch.nn as nn
import cv2
import os
import tqdm.notebook as tq
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from time import time
import matplotlib.pyplot as plt
import random
from glob import glob
import scipy.io
import torch.nn as nn
import numpy as np
from ssqueezepy import cwt
from ssqueezepy.visuals import plot, imshow
import os
import sys
import re
import pandas as pd
from skimage.transform import resize
import warnings
from pathlib import Path 
import pickle

warnings.filterwarnings('ignore')

In [4]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json
api_token = {"username":"****","key":"****"}
import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c alvi-hack-2023

100%|███████████████████████████████████████▉| 229M/229M [00:06<00:00, 38.1MB/s]
100%|████████████████████████████████████████| 229M/229M [00:06<00:00, 35.0MB/s]


In [5]:
!unzip alvi-hack-2023.zip 

Archive:  alvi-hack-2023.zip
  inflating: data_submission/00.npz  
  inflating: data_submission/01.npz  
  inflating: data_submission/02.npz  
  inflating: data_submission/03.npz  
  inflating: data_submission/04.npz  
  inflating: data_submission/05.npz  
  inflating: data_submission/06.npz  
  inflating: data_submission/07.npz  
  inflating: data_submission/08.npz  
  inflating: data_submission/09.npz  
  inflating: data_submission/10.npz  
  inflating: data_submission/11.npz  
  inflating: data_submission/12.npz  
  inflating: data_submission/13.npz  
  inflating: data_submission/14.npz  
  inflating: data_train/00.npz       
  inflating: data_train/01.npz       
  inflating: data_train/02.npz       
  inflating: data_train/03.npz       
  inflating: data_train/04.npz       
  inflating: data_train/05.npz       
  inflating: data_train/06.npz       
  inflating: data_train/07.npz       
  inflating: data_train/08.npz       
  inflating: data_train/09.npz       
  inflating: data_tra

In [20]:
def noisy(image):
    ch, row,col= image.shape
    mean = 0
    var = 0.1
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(ch,row,col))
    gauss = gauss.reshape(ch,row,col)
    noisy = image + gauss
    return noisy

In [25]:
class DataReader(Dataset):
    def __init__(self, path, window, mode='val'):
        self.window = window
        self.path = path
        self.counter = 0
        self.mode = mode
        
        self.x = torch.from_numpy(np.load(path + sorted(os.listdir(path))[0])['data_myo']).type(torch.FloatTensor)
        self.y = torch.from_numpy(np.load(path + sorted(os.listdir(path))[0])['data_vr']).type(torch.FloatTensor)
        for i in sorted(os.listdir(path)):
            if i != sorted(os.listdir(path))[0]:
                self.x = torch.cat([self.x, torch.from_numpy(np.load(path + i)['data_myo']).type(torch.FloatTensor)], dim=0)
                self.y = torch.cat([self.y, torch.from_numpy(np.load(path + i)['data_vr']).type(torch.FloatTensor)], dim=0)
        x_add = self.x[0]
        for i in range(self.window):
            self.x = torch.cat([x_add[None], self.x], dim=0)
        self.x = self.x[:-1]
        
        self.xx = self.x[:200][None]
        n = 1
        self.yy = self.y[0][None]
        while n * self.window + self.window < len(self.y):
            self.xx = torch.cat([self.xx, self.x[n * self.window:n * self.window + self.window ][None]], dim=0)
            self.yy = torch.cat([self.yy, self.y[n * self.window][None]], dim=0)
            n += 1
    def __len__(self):
        return len(self.yy)

    def __getitem__(self, idx):
        # print(f'train - {self.counter}:{self.counter + self.window}')
        # print(f'target - {self.counter}/{len(self.y)}')
        # y = self.y[idx * self.window, :]
        # x = self.x[idx * self.window:idx * self.window + self.window, :]
        x = self.xx[idx]
        y = self.yy[idx]
        x = x.permute(1,0)
        x=convertDF2MNE(x)
        Wx, scales = cwt(x[0], 'morlet')
        x=np.abs(Wx)
        if self.mode == 'train':
            return noisy(resize(x, (8, 224, 224))), y
        else:
            return resize(x, (8, 224, 224)), y

In [7]:
train_path = '/kaggle/working/data_train/'
test_loc = '/kaggle/working/data_test/'

In [8]:
os.mkdir('data_test')
for i in range(10):
    os.replace(train_path+f'{i:02}.npz', test_loc+f'{i:02}.npz')

In [9]:

import mne
def convertDF2MNE(sub):
    info = mne.create_info([str(i) for i in range(len(sub))], ch_types=['eeg'] * len(sub), sfreq=200, verbose=False)
    data=mne.io.RawArray(sub, info, verbose=False)
    data.set_eeg_reference(verbose=False)
    epochs=mne.make_fixed_length_epochs(data,duration=1,overlap=0, verbose=False)
    return epochs._get_data(verbose=False)

In [1]:
train_ds = DataReader(train_path, 200, mode='train')
test_ds = DataReader(test_loc, 200)

train_dataloader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_ds, batch_size=64, shuffle=True)

NameError: name 'DataReader' is not defined

In [11]:
model = torch.hub.load("pytorch/vision", "resnet50", weights="IMAGENET1K_V2")

Using cache found in /root/.cache/torch/hub/pytorch_vision_main


In [ ]:
model.conv1 = nn.Conv2d(8, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

model.fc = nn.Linear(in_features=2048, out_features=64, bias=True)

model

In [ ]:
for param in model.parameters():
    param.requires_grad = False


for param in model.layer4.parameters():
    param.requires_grad = True
for param in model.avgpool.parameters():
    param.requires_grad = True
for param in model.fc.parameters():
    param.requires_grad = True



In [ ]:


def train(model, opt, loss_fn, epochs, saver):
    torch.cuda.empty_cache()
    tr_loss = []
    val_loss = []
    for epoch in tq.tqdm(range(epochs)):
        tic = time()
        print('* Epoch %d/%d' % (epoch+1, epochs))

        tr_avg_loss = 0
        model.train()  # train mode
        train_ds = DataReader(train_path, 200)
        data_tr = DataLoader(train_ds, batch_size=64, shuffle=True)

        for X_batch, Y_batch in tq.tqdm(data_tr):
            X_batch = X_batch.to(device)
            Y_batch = Y_batch.to(device)
            opt.zero_grad()
            Y_pred = model(X_batch)
            Y_pred = Y_pred.view(-1, 16, 4)
            loss = loss_fn(Y_pred, Y_batch)
            loss.backward()
            opt.step()

            tr_avg_loss += loss.cpu().item() / len(data_tr)

        tr_loss.append(tr_avg_loss)
        toc = time()
        print('train loss: %f' % tr_avg_loss)

        avg_loss_val = 0
        model.eval()
        test_ds = DataReader(test_loc, 200)
        data_val = DataLoader(test_ds, batch_size=64, shuffle=False)
        for X_val, Y_val in tq.tqdm(data_val):
            with torch.no_grad():
                Y_hat = model(X_val.to(device)).detach().cpu()
                Y_hat = Y_hat.view(-1, 16, 4)
                loss = loss_fn(Y_hat, Y_val)
                avg_loss_val += loss.item() / len(data_val)

        val_loss.append(avg_loss_val)
        toc = time()
        print('val loss: %f' % avg_loss_val)
        saver.save_best_model(model, avg_loss_val)
        
  

      
    return tr_loss, val_loss



In [ ]:
class BestModel():
    def __init__(self, path, model):
        self.path = path
        self.best_loss = 900
        self.model = model
    def save_best_model(self, model, loss):
        if loss < self.best_loss:
            self.best_loss = loss
            torch.save(self.model, self.path)
            print('model saved')



In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)



In [ ]:
%%time
model = model.to(device)

saver = BestModel('/kaggle/working/resnet.pt', model)
max_epochs = 14
loss = nn.L1Loss()
optim = torch.optim.AdamW(model.parameters(), lr=0.00100, weight_decay=0.05)
tr_loss, val_loss = train(model, optim, loss, max_epochs, saver)

In [ ]:
plt.figure(figsize=(15, 9))
plt.plot(tr_loss, label="train_loss")
plt.plot(val_loss, label="val_loss")
plt.legend(loc='best')
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

In [ ]:
class SubDataReader(Dataset):
    def __init__(self, path, window):
        self.window = window
        self.path = path
        self.x = torch.from_numpy(np.load(path + sorted(os.listdir(path))[0])['data_myo']).type(torch.FloatTensor)
        for i in sorted(os.listdir(path)):
            if i != sorted(os.listdir(path))[0]:
                self.x = torch.cat([self.x, torch.from_numpy(np.load(path + i)['data_myo']).type(torch.FloatTensor)], dim=0)
        x_add = self.x[0]
        for i in range(self.window):
            self.x = torch.cat([x_add[None], self.x], dim=0)
    def __len__(self):
        return len(self.x) - 200

    def __getitem__(self, idx):
        # print(f'train - {self.counter}:{self.counter + self.window}')
        # print(f'target - {self.counter}/{len(self.y)}')
        x = self.x[idx:idx + self.window]
        x = x.permute(1,0)
        x=convertDF2MNE(x)
        Wx, scales = cwt(x[0], 'morlet')
        x=np.abs(Wx)
        
        return resize(x, (8, 224, 224))

In [ ]:
sub_path = '/kaggle/working/data_submission/'

In [ ]:
PATH = '/kaggle/working/resnet.pt'

In [ ]:
model = torch.load(PATH)


In [ ]:
SAVE_PATH = '/kaggle/working/submit.pt'

def sub(model):
    torch.cuda.empty_cache()

    model.eval()
    sub_ds = SubDataReader(sub_path, 200)
    sub_dataloader = DataLoader(sub_ds, batch_size=64, shuffle=True)
    first = True
    for X in tq.tqdm(sub_dataloader):
        with torch.no_grad():
            Y_hat = model(X.to(device)).detach().cpu()
            Y_hat = Y_hat.view(-1, 16, 4)
            if first:
                res = Y_hat
                first = False
            else:
                res = torch.cat([res, Y_hat], dim=0)
            torch.save(res, SAVE_PATH)
            print(res.shape)
            
      
    return res



In [ ]:
DATA_VAL_FOLDER = '/kaggle/working/data_submission/''

In [ ]:
DATA_VAL_FOLDER = Path('/kaggle/working/data_submission/')


In [ ]:
class SubFileDataReader(Dataset):
    def __init__(self, file, window):
        self.window = window
        self.path = file
        self.x = torch.from_numpy(np.load(file)['data_myo']).type(torch.FloatTensor)
        x_add = self.x[0]
        for i in range(self.window):
            self.x = torch.cat([x_add[None], self.x], dim=0)
    def __len__(self):
        return len(self.x) - 200

    def __getitem__(self, idx):
        # print(f'train - {self.counter}:{self.counter + self.window}')
        # print(f'target - {self.counter}/{len(self.y)}')
        x = self.x[idx:idx + self.window]
        x = x.permute(1,0)
        x=convertDF2MNE(x)
        Wx, scales = cwt(x[0], 'morlet')
        x=np.abs(Wx)
        
        return resize(x, (8, 224, 224))

In [ ]:
SAVE_PATH = '/kaggle/working/submit.pt'

def sub_file(model, p):
    torch.cuda.empty_cache()

    model.eval()
    ds = SubFileDataReader(p, 200)
    sub_dataloader = DataLoader(ds, batch_size=64, shuffle=True)
    first = True
    for X in tq.tqdm(sub_dataloader):
        with torch.no_grad():
            Y_hat = model(X.to(device)).detach().cpu()
            Y_hat = Y_hat.view(-1, 16, 4)
            if first:
                res = Y_hat
                first = False
            else:
                res = torch.cat([res, Y_hat], dim=0)
            
      
    return res

In [ ]:
PATH = '/kaggle/working/resnet.pt'
model = torch.load(PATH)


In [ ]:
list_preds = []
model = model.to(device)
print("Start pred")
for p in tq.tqdm(sorted(DATA_VAL_FOLDER.glob('*.npz'))):   
    with torch.no_grad():
        res = sub_file(model, p)
    
    # Put it to common list
    list_preds.append(res)
    print(len(list_preds))

    with open('/kaggle/working/list_preds.pickle', 'wb') as f:
        pickle.dump(list_preds, f)
    

In [ ]:
def prepare_predictions_for_csv(list_predictions):
    """
    [ [Time, 16, 4 ], ... ]
    return np array with N values.  
    """
    result = []
    for pred in list_predictions: 
        pred = np.reshape(pred[::10], [-1])
        result.extend(pred)
    result = np.array(result)
    return result

In [ ]:
dummy_pred = prepare_predictions_for_csv(list_preds)

# There are must be two columns with header Id,Expected
df = pd.DataFrame({'Predicted': dummy_pred})


output_folder = Path(f"/kaggle/working/")
df.to_csv(output_folder / 'submission.csv', index_label = 'Id')